In [3]:
import sys 
sys.path.append(rf"/nfs/home/genovese/thesis-wildfire-genovese/src")
from importlib import reload
import utils
reload(utils)
from utils import *
sys.path.append(rf'/nfs/home/genovese/thesis-wildfire-genovese/src/Nicola_model')
from dataset_with_post import PiedmontDataset

/nfs/home/genovese/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
import gzip
import shutil

# Define the input gzipped file and the output file
input_gz_file = '/nfs/home/genovese/thesis-wildfire-genovese/data/nicola/piedmont.tar.gz'
output_file = '/nfs/home/genovese/thesis-wildfire-genovese/data/nicola/piedmont.tar'

# Open the gzipped file and decompress it
with gzip.open(input_gz_file, 'rb') as f_in:
    with open(output_file, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

print(f"File decompressed to {output_file}")


In [ ]:
import tarfile

# Define the input .tar file and the output directory
input_tar_file = '/nfs/home/genovese/thesis-wildfire-genovese/data/nicola/piedmont.tar'
output_directory = '/nfs/home/genovese/thesis-wildfire-genovese/data/nicola/piedmont/'

# Open the .tar file and extract its contents
with tarfile.open(input_tar_file, 'r') as tar:
    tar.extractall(path=output_directory)

print(f"Files extracted to {output_directory}")


In [ ]:
fires = gpd.read_file('/nfs/home/genovese/thesis-wildfire-genovese/data/nicola/piedmont_2012_2024_fa.geojson')
fires = separate_date(fires, 'initialdate')

In [ ]:
for f in fires[fires.YYYY < 2016].id:
    d = f'/nfs/home/genovese/thesis-wildfire-genovese/database/piedmont/fire_{f}/'
    if os.path.exists(d):
        shutil.rmtree(d)

fires[fires.YYYY >= 2016].to_file('/nfs/home/genovese/thesis-wildfire-genovese/database/piedmont_2016_2024_fa.geojson', index=False, driver='GeoJSON')


In [43]:
gdf = gpd.read_file('/nfs/home/genovese/thesis-wildfire-genovese/database/piedmont_2016_2024_fa.geojson')

In [ ]:
test = PiedmontDataset(root_dir='/nfs/home/genovese/thesis-wildfire-genovese/database/piedmont/',
                       geojson_path = '/nfs/home/genovese/thesis-wildfire-genovese/database/piedmont_2016_2024_fa.geojson')

---

Now it's time to download images for negative fires

In [52]:
gdf.initialdate

0      2016-12-29
1      2016-11-13
2      2016-04-27
3      2017-03-11
4      2017-03-13
          ...    
1161   2023-04-20
1162   2023-04-12
1163   2023-04-17
1164   2023-03-22
1165   2023-03-28
Name: initialdate, Length: 1166, dtype: datetime64[ms]

In [79]:
non_fires = gpd.read_file('/nfs/home/genovese/thesis-wildfire-genovese/data/gathering_geojson/negative_fires_grid.geojson')

In [80]:
def reverse_day(year, day):
    feb = 28
    if year in list(range(2000, 2025, 4)):
        feb += 1
            
    days_per_month = [31, feb, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]

    for index, month in enumerate(days_per_month):
        if (day - month) == 0:
            return (month, index+1)
        elif (day - month) > 0:
            day -= month
        else:
            return (day, index+1)

In [81]:
non_fires['DD'] = non_fires.apply(lambda x: reverse_day(x['YYYY'], x['day'])[0], axis=1)
non_fires['MM'] = non_fires.apply(lambda x: reverse_day(x['YYYY'], x['day'])[1], axis=1)

In [82]:
from pandas._libs.tslibs.timestamps import Timestamp
non_fires['initialdate'] = non_fires.apply(lambda x: Timestamp(year = x.YYYY, month=x.MM, day=x.DD), axis=1)
non_fires['id'] = [-(idx+10) for idx in non_fires.index]

In [83]:
non_fires['polygon'] = non_fires.buffer(1000)

In [84]:
fires = non_fires.set_geometry(col='polygon')

In [87]:
fires.drop(columns='geometry', inplace=True)

In [90]:
fires.rename(columns={'polygon': 'geometry'}, inplace=True)

In [91]:
fires.to_file('/nfs/home/genovese/thesis-wildfire-genovese/src/OnFireForest/sentinel_negative_fires/negatives.geojson',driver='GeoJSON', index=False)

---

In [107]:
f = gpd.read_file('/nfs/home/genovese/thesis-wildfire-genovese/src/OnFireForest/sentinel_negative_fires/negatives.geojson').to_crs('32632')

In [109]:
f.to_file('/nfs/home/genovese/thesis-wildfire-genovese/src/OnFireForest/sentinel_negative_fires/negatives.geojson',driver='GeoJSON', index=False)

In [110]:
sys.path.append(rf'/nfs/home/genovese/thesis-wildfire-genovese/src/Nicola_model')
import generate_sentinel_new
reload(generate_sentinel_new)
from generate_sentinel_new import *

In [ ]:
config_path = "/nfs/home/genovese/thesis-wildfire-genovese/src/OnFireForest/sentinel_negative_fires/congif.yaml" # Definisci il percorso del config

config_example = {
        "geojson_path": "/nfs/home/genovese/thesis-wildfire-genovese/src/OnFireForest/sentinel_negative_fires/negatives.geojson",
        "interval_pre_fire_days": 7, # AGGIORNATO
        "satellite": "sentinel-2",  # AGGIORNATO
        "patch_size_pixels": 256,
        "target_crs": "EPSG:32632", # NUOVO
        "min_fire_area_sq_m": 200,  # NUOVO
        "image_similarity_tolerance": 0.005 # NUOVO
    }

with open(config_path, "w") as f:
    yaml.dump(config_example, f)
    
print(f"Generato o aggiornato il file di configurazione: {config_path}")


config_data = load_config(config_path) # Carica il config dal percorso specificato

print("\n--- Inizio Processo Batch per TUTTI gli incendi ---")

gdf_all_fires_filtered = gpd.read_file(config_data["geojson_path"])
min_date_for_sentinel2 = datetime(2015, 6, 23) 
    
# # Filtra gli incendi per data (solo quelli dopo l'inizio di Sentinel-2)
# gdf_all_fires_filtered = gdf_all_fires[pd.to_datetime(gdf_all_fires["initialdate"]) >= min_date_for_sentinel2]
    
print(f"Trovati {len(gdf_all_fires_filtered)} incendi nel GeoJSON. Filtrati {len(gdf_all_fires_filtered)} incendi dopo il {min_date_for_sentinel2.strftime('%Y-%m-%d')}.")

for idx, fire_row in tqdm(gdf_all_fires_filtered.iterrows()):
    try:
        process_single_fire(fire_row["id"], config_data, '/nfs/home/genovese/thesis-wildfire-genovese/database/negative_piedmont')
    except Exception as e:
        print(f"🚨 ERRORE GRAVE durante il processo dell'incendio ID {fire_row['id']}: {e}")
            # Puoi aggiungere qui della logica per registrare gli errori o riprovare

print("\n--- Processo Batch Completato ---")

Generato o aggiornato il file di configurazione: /nfs/home/genovese/thesis-wildfire-genovese/src/OnFireForest/sentinel_negative_fires/congif.yaml

--- Inizio Processo Batch per TUTTI gli incendi ---
Trovati 1086 incendi nel GeoJSON. Filtrati 1086 incendi dopo il 2015-06-23.


0it [00:00, ?it/s]

Caricamento GeoJSON da: /nfs/home/genovese/thesis-wildfire-genovese/src/OnFireForest/sentinel_negative_fires/negatives.geojson

--- Processando Incendio ID: -10 ---
Data dell'incendio: 2020-05-03
Intervallo di ricerca STAC (PRE-incendio): 2020-04-26/2020-05-03
Dimensione patch target (finale): 256x256 pixel (2.56 km per lato).
Cartella di output: /nfs/home/genovese/thesis-wildfire-genovese/database/negative_piedmont/fire_-10
Tolleranza di somiglianza immagini: 0.500%
Ricerca STAC con bbox (WGS84): (8.136865294187722, 44.74111651473986, 8.187136638935385, 44.77689102879712)
✅ Trovate 3 immagini SENTINEL-2 nell'intervallo PRE-incendio.
  Immagine 1 del 2020-05-01 interseca la GT. Aggiunta alla lista.
  Immagine 2 del 2020-04-29 interseca la GT. Aggiunta alla lista.
  Immagine 3 del 2020-04-26 interseca la GT. Aggiunta alla lista.

✅ Iniziamo l'elaborazione di 3 immagini valide.

📸 Elaborazione immagine 1/3: S2A_MSIL2A_20200501T102031_R065_T32TMQ_20200920T011538 del 2020-05-01.
💾 Salvato 

1it [00:06,  6.86s/it]

💾 Salvato immagine Sentinel (TIFF multi-banda): /nfs/home/genovese/thesis-wildfire-genovese/database/negative_piedmont/fire_-10/fire_-10_2020-04-26_pre_sentinel_3.tif

--- FINE Processo per incendio ID: -10 ---
Caricamento GeoJSON da: /nfs/home/genovese/thesis-wildfire-genovese/src/OnFireForest/sentinel_negative_fires/negatives.geojson

--- Processando Incendio ID: -11 ---
Data dell'incendio: 2022-12-31
Intervallo di ricerca STAC (PRE-incendio): 2022-12-24/2022-12-31
Dimensione patch target (finale): 256x256 pixel (2.56 km per lato).
Cartella di output: /nfs/home/genovese/thesis-wildfire-genovese/database/negative_piedmont/fire_-11
Tolleranza di somiglianza immagini: 0.500%
Ricerca STAC con bbox (WGS84): (8.015538909497485, 45.14478764974714, 8.066037047011811, 45.18047073070988)
✅ Trovate 6 immagini SENTINEL-2 nell'intervallo PRE-incendio.
  Immagine 1 del 2022-12-30 interseca la GT. Aggiunta alla lista.
  ❌ Immagine 2 del 2022-12-30 - Non interseca la geometria GT.
  Immagine 3 del 2

2it [00:15,  7.86s/it]

💾 Salvato immagine Sentinel (TIFF multi-banda): /nfs/home/genovese/thesis-wildfire-genovese/database/negative_piedmont/fire_-11/fire_-11_2022-12-25_pre_sentinel_3.tif

--- FINE Processo per incendio ID: -11 ---
Caricamento GeoJSON da: /nfs/home/genovese/thesis-wildfire-genovese/src/OnFireForest/sentinel_negative_fires/negatives.geojson

--- Processando Incendio ID: -12 ---
Data dell'incendio: 2020-12-22
Intervallo di ricerca STAC (PRE-incendio): 2020-12-15/2020-12-22
Dimensione patch target (finale): 256x256 pixel (2.56 km per lato).
Cartella di output: /nfs/home/genovese/thesis-wildfire-genovese/database/negative_piedmont/fire_-12
Tolleranza di somiglianza immagini: 0.500%
Ricerca STAC con bbox (WGS84): (8.32637672970031, 45.9659590997027, 8.377353711039211, 46.00145588083301)
✅ Trovate 8 immagini SENTINEL-2 nell'intervallo PRE-incendio.
  Immagine 1 del 2020-12-22 interseca la GT. Aggiunta alla lista.
  Immagine 2 del 2020-12-22 interseca la GT. Aggiunta alla lista.
  Immagine 3 del 